In [1]:
import numpy as np
import pandas as pd
import json
import re
import requests
import time
import urllib
from bs4 import BeautifulSoup
from bs4.element import NavigableString, Tag, Comment

In [2]:
def get_page(url):
    response = requests.get(url)
    page = response.text
    return page

def get_soup(url):
    page = get_page(url)
    soup = BeautifulSoup(page, "html.parser")
    return soup

def remove_parentheses(text):
    """
    Remove all blocks of text wrapped in parentheses or brackets, squeeze the spaces and strip the text.
    """
    reP = '(\[[^\]]*\]|\([^\)]*\))'
    reS = '\s+'
    return re.sub(reS, ' ', re.sub(reP, ' ', text.replace('\xa0',' '))).strip()

# Parcer 1

In [110]:
# Version 1

# url = 'http://giveadaygetadisneyday.blogspot.com/2014/12/peter-pan-1953-script.html'

def url_parcer_1_v1(url):
    soup = get_soup(url)
    div = soup.find('div', {'class': 'entry-content'})
    script = div.get_text()
    list_script = script.split('\n')

    # Remove text in brackets
    no_parentheses = []
    for line in list_script:
        line = remove_parentheses(line)
        no_parentheses.append(line)
        
    # Remove blank lines
    new_list = []
    for line in no_parentheses:
        if line != '':
            new_list.append(line)    
    new_list = new_list[1:new_list.index('THE END')]

    pattern = re.compile('^[A-Z]+\s*.*?:')
    
    df = pd.DataFrame({'Character': [], 'Speech': []})
    df["Character"] = df["Character"].astype(str)
    df["Speech"] = df["Speech"].astype(str)
    
    # Filling dataframe
    counter = 0

    for line in new_list:
    
        if pattern.match(line):
            find_character = re.findall('^[A-Z]+\s*.*?:', line)
            character = find_character[0].lower()[:-1]
        
        
            if character == 'smee well, captain, it’s nice to see you smiling again brings back the good old days when we was leading an healthy, normal life':
                df.at[counter, 'Character'] = 'smee'
                new_line = re.findall('^[A-Z]+\s*.*?:\s*(.*\s*)', line)
                df.at[counter, 'Speech'] = character[4:] + new_line[0]
            
        
            elif character == 'peter':
                df.at[counter, 'Character'] = 'peter pan'
                new_line = re.findall('^[A-Z]+\s*.*?:\s*(.*\s*)', line)
                df.at[counter, 'Speech'] = new_line[0]
            
            
            else:
                df.at[counter, 'Character'] = character
                df.at[counter, 'Speech'] = re.findall('^[A-Z]+\s*.*?:\s*(.*\s*)', line)[0]
        else:
            df.at[counter, 'Character'] = character
            df.at[counter, 'Speech'] = line
            
        counter +=1
        
    return df

In [111]:
df = url_parcer_1_v1('http://giveadaygetadisneyday.blogspot.com/2014/12/peter-pan-1953-script.html')

In [112]:
df

,Character,Speech
0,choir,The second star to the right shines in the nig...
1,choir,To tell you that the dreams you plan really ca...
2,choir,The second star to the right shines with a lig...
3,choir,And if it's Never Land you need it's light wil...
4,choir,"Twinkle, twinkle little star so we'll know whe..."
...,...,...
868,off singing voice,there's no better time to start
869,off singing voice,think of all the joy you'll find
870,off singing voice,when you leave the world behind
871,off singing voice,and bid your cares good-bye


In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 873 entries, 0 to 872
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Character  873 non-null    object
 1   Speech     873 non-null    object
dtypes: object(2)
memory usage: 60.5+ KB


In [114]:
# Version 2

def url_parcer_1_v2(url):
    soup = get_soup(url)
    div = soup.find('div', {'class': 'entry-content'})
    script = div.get_text()
    list_script = script.split('\n')

    # Remove text in brackets
    no_parentheses = []
    for line in list_script:
        line = remove_parentheses(line)
        no_parentheses.append(line)
        
    # Remove blank lines
    new_list = []
    for line in no_parentheses:
        if line != '':
            new_list.append(line)    
    new_list = new_list[1:new_list.index('THE END')]

    pattern = re.compile('^[A-Z]+\s*.*?:')
    
    df = pd.DataFrame({'Character': [], 'Speech': []})
    df["Character"] = df["Character"].astype(str)
    df["Speech"] = df["Speech"].astype(str)
    
    # Filling dataframe Version 2
    counter = 0

    for line in new_list:
    
        if pattern.match(line):
            find_character = re.findall('^[A-Z]+\s*.*?:', line)
            character = find_character[0].lower()[:-1]
        
        
            if character == 'smee well, captain, it’s nice to see you smiling again brings back the good old days when we was leading an healthy, normal life':
                df.at[counter, 'Character'] = 'smee'
                new_line = re.findall('^[A-Z]+\s*.*?:(\s*.*\s*)', line)
                df.at[counter, 'Speech'] = character[4:] + new_line[0]
            
        
            elif character == 'peter':
                df.at[counter, 'Character'] = 'peter pan'
                new_line = re.findall('^[A-Z]+\s*.*?:\s*(.*\s*)', line)
                df.at[counter, 'Speech'] = new_line[0]
            
            
            else:
                df.at[counter, 'Character'] = character
                df.at[counter, 'Speech'] = re.findall('^[A-Z]+\s*.*?:\s*(.*\s*)', line)[0]
        else:
            index = df[df['Character'] == character].index[0]
            line = ' ' + line
            df.at[index, 'Speech'] += line
            
        counter +=1
        
    return df

In [115]:
df = url_parcer_1_v2('http://giveadaygetadisneyday.blogspot.com/2014/12/peter-pan-1953-script.html')

In [116]:
df

,Character,Speech
0,choir,The second star to the right shines in the nig...
9,narrator,"All this has happened before, and it will all ..."
10,mrs. darling,"George, dear, do hurry. We mustn’t be late for..."
11,narrator,Mrs. Darling believed that Peter Pan was the s...
12,mr. darling,"Mary, unless I find my cuff links we don’t go ..."
...,...,...
863,mrs. darling,George. George!
864,mr. darling,"Now what, Mary? gasps Nana, d-did you see– You..."
865,mrs. darling,"George, dear."
866,wendy,Father.


In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 628 entries, 0 to 867
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Character  628 non-null    object
 1   Speech     628 non-null    object
dtypes: object(2)
memory usage: 34.7+ KB


# Parcer 2

In [127]:
# http://www.fpx.de/fp/Disney/Scripts/SleepingBeauty/sb.html

def parcer_2(url):
    
    soup = get_soup(url)
    script = soup.get_text()  
    list_script = script.split('\n')
    
    # Remove text in brackets
    no_parentheses = []
    for line in list_script:
        line = remove_parentheses(line)
        no_parentheses.append(line)
        
    # Remove blank lines
    new_list = []
    for line in no_parentheses:
        if line != '':
            new_list.append(line)  
    new_list = new_list[new_list.index('Narrator: In a far away land, long ago,') : new_list.index('The End')]
    
    pattern = re.compile('^[A-Z0-9]+\s*.*?:')
    
    df = pd.DataFrame({'Character': [], 'Speech': []})
    df["Character"] = df["Character"].astype(str)
    df["Speech"] = df["Speech"].astype(str)
    
    # Filling dataframe
    counter = 0

    for line in new_list:
    
        if pattern.match(line):
            character = re.findall('^[A-Z0-9]+\s*.*?:', line)[0][:-1]
        
            if character == 'You said so yourself':
                character = 'Phillip'
                df.at[counter, 'Character'] = character
                df.at[counter, 'Speech'] = line
            
            
            else:
                df.at[counter, 'Character'] = character
                df.at[counter, 'Speech'] = re.findall('^[A-Z0-9]+\s*.*?:\s*(.*\s*)', line)[0]
        
        
        else:
            df.at[counter, 'Character'] = character
            df.at[counter, 'Speech'] = line
            
        counter +=1

    
    df = df[df['Speech']!='']
    
    
    return df

In [128]:
df = parcer_2('http://www.fpx.de/fp/Disney/Scripts/SleepingBeauty/sb.html')

In [129]:
df

,Character,Speech
0,Narrator,"In a far away land, long ago,"
1,Narrator,lived a king and his fair queen.
2,Narrator,Many years had they longed for a child
3,Narrator,and finally their wish was granted.
4,Narrator,"A daughter was born, and they called her Aurora."
...,...,...
896,Choir,That visions are seldom all they seem
897,Choir,But if I know you I know what you'll do
898,Choir,You'll love me at once
899,Choir,The way you did


In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 877 entries, 0 to 900
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Character  877 non-null    object
 1   Speech     877 non-null    object
dtypes: object(2)
memory usage: 20.6+ KB


In [131]:
df['Character'].unique()

array(['Narrator', 'Choir', 'Announcer', 'Fairies', 'Flora', 'Fauna',
       'Merryweather', 'Maleficent', 'Queen', 'Stefan', '1st servant',
       '2nd servant', 'Servants', 'Briar Rose', 'Phillip', 'Owl', 'Both',
       'Hubert'], dtype=object)